# Diagrama UML

Libro
- id (PK)
- titulo
- precio
- rating
- in_stock
- categoria_id (FK)

Autor
- id (PK)
- nombre

Libro_Autor
- libro_id (FK)
- autor_id (FK)

Categoria
- id (PK)
- nombre

# Esquema SQL

## DDL

CREATE TABLE Categoria (   
- id SERIAL PRIMARY KEY,  
- nombre TEXT NOT NULL  
);


CREATE TABLE Autor (  
- id SERIAL PRIMARY KEY,  
- nombre TEXT NOT NULL  
);  

CREATE TABLE Libro (  
- id SERIAL PRIMARY KEY,  
- titulo TEXT NOT NULL,  
- precio NUMERIC(6, 2) NOT NULL,  
- rating INTEGER NOT NULL,  
- in_stock BOOLEAN NOT NULL,  
- categoria_id INTEGER NOT NULL,  
- FOREIGN KEY (categoria_id) REFERENCES Categoria(id)  
);  

CREATE TABLE Libro_Autor (  
- libro_id INTEGER NOT NULL,  
- autor_id INTEGER NOT NULL,  
- PRIMARY KEY (libro_id, autor_id),  
- FOREIGN KEY (libro_id) REFERENCES Libro(id),  
- FOREIGN KEY (autor_id) REFERENCES Autor(id)  
);

In [4]:
%pip install requests
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Using cached soupsieve-2.7-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://books.toscrape.com/'

# obtengo el html crudo
response = requests.get(url)

# separar por etiquetas (metafora sopa de etiquetas)
soup = BeautifulSoup(response.text, 'html.parser')

# Extraer los libros en la página donde haya un article con class product_pod
libros = soup.select('article.product_pod')

# Mostrar los primeros 3 títulos como prueba
for libro in libros[:3]:
    titulo = libro.h3.a['title']
    print(titulo)

A Light in the Attic
Tipping the Velvet
Soumission
